In [5]:
import pandas as pd
import tabula
import numpy as np

In [6]:
lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

vecteur = [lieux[0] if i < 4 else
           lieux[1] if i < 8 else
           lieux[2] if i < 13 else
           lieux[3] if i < 15 else
           lieux[4] if i < 17 else
           lieux[5] if i < 19 else
           lieux[6] if i < 29 else
           lieux[7] if i < 35 else
           lieux[8] if i < 48 else
           lieux[9] if i < 58 else
           lieux[10] if i < 63 else
           lieux[11] if i < 68 else
           lieux[12] if i < 74 else
           lieux[13] if i < 78 else
           lieux[14] if i < 79 else
           lieux[15] if i < 83 else
           'Valeur par défaut' for i in range(83)]


# Lien vers le PDF contenant le tableau
lien_pdf = "https://medias.paris2024.org/uploads/2022/07/Calendrier-par-epreuves-des-Jeux-Olympiques-de-Paris-2024.pdf"

try : 
    # Utilisation de tabula pour extraire le tableau du PDF
    # Utilisation de l'option 'pages' pour spécifier la page ou les pages contenant le tableau
    df_list = tabula.read_pdf(lien_pdf, pages=[1, 2], stream=True)  # Adapter les options selon vos besoins

    # Concaténer les DataFrames obtenus des différentes pages en un seul DataFrame
    df = pd.concat(df_list)

    # Supprimer les 28 dernières lignes
    df = df.iloc[:-32]
    df.drop(24, inplace=True)
    df.drop(23, inplace=True)
    df.drop(55, inplace=True)

    # Supprimer la première ligne du DataFrame
    df = df.drop(df.index[[0, 2]])  # Suppression de la première ligne
    df = df.drop(df.columns[[0]], axis=1)

    # Supprimer les lignes contenant uniquement des NaN
    df = df.dropna(how='all')
    
    df.columns = df.iloc[0]
    df.drop(1, inplace=True)

    df.index = vecteur
    
    df = df.replace('<', np.nan)

except Exception as e:
        print("Une erreur s'est produite :", str(e))




# %%
def format_date_horaire(cell, index_value):
    if isinstance(cell, str) and '-' in cell:
        date_month = '07' if int(index_value) in range(24, 32) else '08'
        year = '2024'
        
        date = f"{index_value}/{date_month}/{year}"
        horaire1, horaire2 = cell.split('-')
        
        return f"{date} {horaire1.strip()} - {horaire2.strip()}"
    else:
        return cell

# Appliquer la fonction à chaque cellule du DataFrame
for column in df.columns:
    df[column] = df.apply(lambda x: format_date_horaire(x[column], column), axis=1)


# %%
# Supprimer les titres des index
df.index.name = None

# Ajouter Lieux pour la première colonne
df.reset_index(inplace=True)
df.columns = ['Lieux'] + list(df.columns[1:])  # Renommer la première colonne en 'Lieux'

# %%
# Créer un dictionnaire pour stocker les valeurs non vides associées à chaque lieu
data = {}
for idx, row in df.iterrows():
    lieu = row['Lieux']
    if lieu not in data:
        data[lieu] = []

    # Récupérer les valeurs non vides et les ajouter à la liste des valeurs du lieu
    values = row.drop('Lieux').dropna().tolist()
    data[lieu].extend(values)

# Créer un nouveau DataFrame à partir du dictionnaire
new_df = pd.DataFrame.from_dict(data, orient='index')

lieux_a_supprimer = ['arena-paris-nord', 'site-escalade-bourget', 'centre-aquatique']

# Suppression des lignes spécifiées
new_df = new_df.drop(lieux_a_supprimer, axis=0)

# %%
# Fonction pour convertir le format de date
def convert_date_format(entry):
    if isinstance(entry, str):
        parts = entry.split(' ')
        if len(parts) == 5:
            date = parts[0]
            start_time = parts[1]
            return f"{date} {start_time}"
    return entry

# Appliquer la fonction sur toutes les colonnes du DataFrame
for col in new_df.columns:
    new_df[col] = new_df[col].apply(convert_date_format)


# %%
# Appliquer une fonction pour supprimer la partie après le tiret dans chaque cellule
horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)

# Appliquer une fonction pour supprimer les 4 caractères avant le tiret dans chaque cellule
horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)

horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_début = horaires_début.dropna(axis=1, how='all')

horaires_fin = horaires_fin.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_fin = horaires_fin.dropna(axis=1, how='all')


# %%
def adjust_time_moins_90min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Ajouter 1 heure et 30 minutes à l'objet datetime
        cell = cell - pd.Timedelta(hours=1, minutes=30)
    return cell

def adjust_time_plus_30min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Retirer 30 minutes à l'objet datetime
        cell = cell + pd.Timedelta(hours=0, minutes=30)
    return cell

# Parcourir chaque cellule du DataFrame et appliquer la fonction adjust_time
new_horaires_début_df = horaires_début.map(adjust_time_moins_90min)
new_horaires_fin_df = horaires_fin.map(adjust_time_plus_30min)

new_horaires_début_df.reset_index(inplace=True)
new_horaires_début_df.rename(columns={'index': 'Lieu'}, inplace=True)

new_horaires_fin_df.reset_index(inplace=True)
new_horaires_fin_df.rename(columns={'index': 'Lieu'}, inplace=True)

print("horaires du début des évènements")
display(new_horaires_début_df)
print("horaires de la fin des évènements")
display(new_horaires_fin_df)

new_horaires_fin_df.to_excel('new_horaires_fin_df.xlsx', index=False)
new_horaires_début_df.to_excel('new_horaires_début_df.xlsx', index=False)

# %%
def ajuster_plage_horaire(heure):
    if pd.notnull(heure):
        # Extraire la date et l'heure de la cellule
        date, time = str(heure).split(' ')
        heure, minute, seconde = map(int, time.split(':')[0:3])
        
        # Calculer la plage horaire correspondante
        plage_horaire = f"{heure:02d}:00:00 - {(heure+1)%24:02d}:00:00"
        
        # Construire la date avec la plage horaire
        return f"{date} {plage_horaire}"
    
    return heure

# Appliquer la fonction à toutes les colonnes (sauf la première)
new_horaires_début_df.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].map(ajuster_plage_horaire)
new_horaires_fin_df.iloc[:, 1:] = new_horaires_fin_df.iloc[:, 1:].map(ajuster_plage_horaire)

print("tranches horaire du début des évènements")
display(new_horaires_début_df)
print("tranches horaire de la fin des évènements")
display(new_horaires_fin_df)

/tmp/ipykernel_7559/898600952.py:140: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)
/tmp/ipykernel_7559/898600952.py:143: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)
/tmp/ipykernel_7559/898600952.py:145: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
/tmp/ipykernel_7559/898600952.py:145: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
/tmp/ipykernel_7559/898600952.py:148: 

horaires du début des évènements


,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 07:30:00,2024-07-29 07:30:00,2024-07-30 07:30:00,2024-07-31 07:30:00,2024-01-08 07:30:00,2024-02-08 07:30:00,2024-03-08 07:30:00,2024-04-08 07:30:00,2024-05-08 07:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 08:30:00,2024-07-28 08:30:00,2024-07-29 08:30:00,2024-07-30 08:30:00,2024-07-31 08:30:00,2024-01-08 08:30:00,2024-02-08 08:30:00,2024-03-08 06:30:00,2024-07-27 14:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 08:30:00,2024-07-28 08:00:00,2024-07-29 08:00:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 08:30:00,2024-02-08 10:30:00,2024-03-08 10:00:00,2024-04-08 08:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 08:00:00,2024-07-28 08:00:00,2024-07-29 08:00:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 08:00:00,2024-02-08 08:00:00,2024-03-08 08:00:00,2024-04-08 08:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 06:30:00,2024-07-31 06:30:00,2024-05-08 06:30:00,2024-08-08 06:00:00,2024-09-08 06:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 06:00:00,2024-03-08 09:30:00,2024-04-08 12:30:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 07:30:00,2024-02-08 07:30:00,2024-01-08 11:00:00,2024-07-30 16:00:00,2024-07-31 16:00:00,2024-01-08 16:30:00,2024-02-08 16:00:00,2024-03-08 16:00:00,2024-04-08 16:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 16:00:00,2024-07-08 16:00:00,2024-07-27 10:30:00,2024-07-28 10:30:00,2024-07-29 10:30:00,2024-07-30 10:30:00,2024-07-31 10:30:00,2024-01-08 10:30:00,2024-02-08 10:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-08 20:00:00,2024-07-08 20:00:00,2024-08-08 20:00:00,2024-09-08 20:00:00,2024-10-08 20:00:00,2024-07-27 07:30:00,2024-07-28 07:30:00,2024-07-29 07:30:00,2024-07-30 07:30:00,...,2024-10-08 10:00:00,2024-11-08 10:00:00,2024-07-08 13:30:00,2024-08-08 13:30:00,2024-09-08 13:30:00,2024-10-08 14:30:00,2024-07-08 18:00:00,2024-08-08 18:00:00,2024-09-08 18:00:00,2024-10-08 19:00:00
9,arena-bercy,2024-07-27 09:30:00,2024-07-28 08:00:00,2024-07-28 13:20:00,2024-05-08 10:30:00,2024-07-27 14:00:00,2024-07-28 16:30:00,2024-03-08 14:00:00,2024-04-08 13:30:00,2024-07-27 18:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


horaires de la fin des évènements


,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 13:30:00,2024-07-29 13:30:00,2024-07-30 13:30:00,2024-07-31 13:30:00,2024-01-08 13:30:00,2024-02-08 13:30:00,2024-03-08 13:30:00,2024-04-08 11:30:00,2024-05-08 11:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 14:30:00,2024-07-28 14:30:00,2024-07-29 14:30:00,2024-07-30 14:30:00,2024-07-31 14:30:00,2024-01-08 14:30:00,2024-02-08 14:30:00,2024-03-08 14:30:00,2024-07-27 19:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 17:10:00,2024-07-28 17:20:00,2024-07-29 17:20:00,2024-07-30 18:00:00,2024-07-31 18:00:00,2024-01-08 17:40:00,2024-02-08 18:00:00,2024-03-08 17:30:00,2024-04-08 17:40:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 13:00:00,2024-07-28 11:35:00,2024-07-29 11:35:00,2024-07-30 16:25:00,2024-07-31 16:25:00,2024-01-08 13:55:00,2024-02-08 12:35:00,2024-03-08 11:45:00,2024-04-08 11:45:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 11:00:00,2024-07-31 11:10:00,2024-05-08 10:40:00,2024-08-08 11:00:00,2024-09-08 11:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 12:15:00,2024-03-08 18:45:00,2024-04-08 19:15:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 11:30:00,2024-02-08 11:30:00,2024-01-08 15:00:00,2024-07-30 20:00:00,2024-07-31 20:00:00,2024-01-08 20:30:00,2024-02-08 20:00:00,2024-03-08 20:00:00,2024-04-08 20:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 19:30:00,2024-07-08 19:30:00,2024-07-27 17:30:00,2024-07-28 17:30:00,2024-07-29 17:30:00,2024-07-30 17:30:00,2024-07-31 17:30:00,2024-01-08 17:30:00,2024-02-08 17:30:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-09 00:00:00,2024-07-09 00:00:00,2024-08-09 00:00:00,2024-09-09 00:00:00,2024-10-09 00:00:00,2024-07-27 12:00:00,2024-07-28 12:00:00,2024-07-29 12:00:00,2024-07-30 12:00:00,...,2024-10-08 14:30:00,2024-11-08 14:30:00,2024-07-08 18:00:00,2024-08-08 18:00:00,2024-09-08 18:00:00,2024-10-08 19:00:00,2024-07-08 22:30:00,2024-08-08 22:30:00,2024-09-08 22:30:00,2024-10-08 23:30:00
9,arena-bercy,2024-07-27 14:00:00,2024-07-28 13:50:00,2024-07-28 17:00:00,2024-05-08 15:45:00,2024-07-27 18:30:00,2024-07-28 20:10:00,2024-03-08 18:30:00,2024-04-08 17:55:00,2024-07-27 23:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


tranches horaire du début des évènements


,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 07:00:00 - 08:00:00,2024-07-29 07:00:00 - 08:00:00,2024-07-30 07:00:00 - 08:00:00,2024-07-31 07:00:00 - 08:00:00,2024-01-08 07:00:00 - 08:00:00,2024-02-08 07:00:00 - 08:00:00,2024-03-08 07:00:00 - 08:00:00,2024-04-08 07:00:00 - 08:00:00,2024-05-08 07:00:00 - 08:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 08:00:00 - 09:00:00,2024-07-28 08:00:00 - 09:00:00,2024-07-29 08:00:00 - 09:00:00,2024-07-30 08:00:00 - 09:00:00,2024-07-31 08:00:00 - 09:00:00,2024-01-08 08:00:00 - 09:00:00,2024-02-08 08:00:00 - 09:00:00,2024-03-08 06:00:00 - 07:00:00,2024-07-27 14:00:00 - 15:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 08:00:00 - 09:00:00,2024-07-28 08:00:00 - 09:00:00,2024-07-29 08:00:00 - 09:00:00,2024-07-30 10:00:00 - 11:00:00,2024-07-31 10:00:00 - 11:00:00,2024-01-08 08:00:00 - 09:00:00,2024-02-08 10:00:00 - 11:00:00,2024-03-08 10:00:00 - 11:00:00,2024-04-08 08:00:00 - 09:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 08:00:00 - 09:00:00,2024-07-28 08:00:00 - 09:00:00,2024-07-29 08:00:00 - 09:00:00,2024-07-30 10:00:00 - 11:00:00,2024-07-31 10:00:00 - 11:00:00,2024-01-08 08:00:00 - 09:00:00,2024-02-08 08:00:00 - 09:00:00,2024-03-08 08:00:00 - 09:00:00,2024-04-08 08:00:00 - 09:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 06:00:00 - 07:00:00,2024-07-31 06:00:00 - 07:00:00,2024-05-08 06:00:00 - 07:00:00,2024-08-08 06:00:00 - 07:00:00,2024-09-08 06:00:00 - 07:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 06:00:00 - 07:00:00,2024-03-08 09:00:00 - 10:00:00,2024-04-08 12:00:00 - 13:00:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 07:00:00 - 08:00:00,2024-02-08 07:00:00 - 08:00:00,2024-01-08 11:00:00 - 12:00:00,2024-07-30 16:00:00 - 17:00:00,2024-07-31 16:00:00 - 17:00:00,2024-01-08 16:00:00 - 17:00:00,2024-02-08 16:00:00 - 17:00:00,2024-03-08 16:00:00 - 17:00:00,2024-04-08 16:00:00 - 17:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 16:00:00 - 17:00:00,2024-07-08 16:00:00 - 17:00:00,2024-07-27 10:00:00 - 11:00:00,2024-07-28 10:00:00 - 11:00:00,2024-07-29 10:00:00 - 11:00:00,2024-07-30 10:00:00 - 11:00:00,2024-07-31 10:00:00 - 11:00:00,2024-01-08 10:00:00 - 11:00:00,2024-02-08 10:00:00 - 11:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-08 20:00:00 - 21:00:00,2024-07-08 20:00:00 - 21:00:00,2024-08-08 20:00:00 - 21:00:00,2024-09-08 20:00:00 - 21:00:00,2024-10-08 20:00:00 - 21:00:00,2024-07-27 07:00:00 - 08:00:00,2024-07-28 07:00:00 - 08:00:00,2024-07-29 07:00:00 - 08:00:00,2024-07-30 07:00:00 - 08:00:00,...,2024-10-08 10:00:00 - 11:00:00,2024-11-08 10:00:00 - 11:00:00,2024-07-08 13:00:00 - 14:00:00,2024-08-08 13:00:00 - 14:00:00,2024-09-08 13:00:00 - 14:00:00,2024-10-08 14:00:00 - 15:00:00,2024-07-08 18:00:00 - 19:00:00,2024-08-08 18:00:00 - 19:00:00,2024-09-08 18:00:00 - 19:00:00,2024-10-08 19:00:00 - 20:00:00
9,arena-bercy,2024-07-27 09:00:00 - 10:00:00,2024-07-28 08:00:00 - 09:00:00,2024-07-28 13:00:00 - 14:00:00,2024-05-08 10:00:00 - 11:00:00,2024-07-27 14:00:00 - 15:00:00,2024-07-28 16:00:00 - 17:00:00,2024-03-08 14:00:00 - 15:00:00,2024-04-08 13:00:00 - 14:00:00,2024-07-27 18:00:00 - 19:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


tranches horaire de la fin des évènements


,Lieu,0,1,2,3,4,5,6,7,8,...,112,113,114,115,116,117,118,119,120,121
0,stade-tour-eiffel,2024-07-28 13:00:00 - 14:00:00,2024-07-29 13:00:00 - 14:00:00,2024-07-30 13:00:00 - 14:00:00,2024-07-31 13:00:00 - 14:00:00,2024-01-08 13:00:00 - 14:00:00,2024-02-08 13:00:00 - 14:00:00,2024-03-08 13:00:00 - 14:00:00,2024-04-08 11:00:00 - 12:00:00,2024-05-08 11:00:00 - 12:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,arena-champ-de-mars,2024-07-27 14:00:00 - 15:00:00,2024-07-28 14:00:00 - 15:00:00,2024-07-29 14:00:00 - 15:00:00,2024-07-30 14:00:00 - 15:00:00,2024-07-31 14:00:00 - 15:00:00,2024-01-08 14:00:00 - 15:00:00,2024-02-08 14:00:00 - 15:00:00,2024-03-08 14:00:00 - 15:00:00,2024-07-27 19:00:00 - 20:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,grand-palais,2024-07-27 17:00:00 - 18:00:00,2024-07-28 17:00:00 - 18:00:00,2024-07-29 17:00:00 - 18:00:00,2024-07-30 18:00:00 - 19:00:00,2024-07-31 18:00:00 - 19:00:00,2024-01-08 17:00:00 - 18:00:00,2024-02-08 18:00:00 - 19:00:00,2024-03-08 17:00:00 - 18:00:00,2024-04-08 17:00:00 - 18:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,invalides,2024-07-25 13:00:00 - 14:00:00,2024-07-28 11:00:00 - 12:00:00,2024-07-29 11:00:00 - 12:00:00,2024-07-30 16:00:00 - 17:00:00,2024-07-31 16:00:00 - 17:00:00,2024-01-08 13:00:00 - 14:00:00,2024-02-08 12:00:00 - 13:00:00,2024-03-08 11:00:00 - 12:00:00,2024-04-08 11:00:00 - 12:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,pont-alexandre-iii,2024-07-30 11:00:00 - 12:00:00,2024-07-31 11:00:00 - 12:00:00,2024-05-08 10:00:00 - 11:00:00,2024-08-08 11:00:00 - 12:00:00,2024-09-08 11:00:00 - 12:00:00,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,trocadero,2024-01-08 12:00:00 - 13:00:00,2024-03-08 18:00:00 - 19:00:00,2024-04-08 19:00:00 - 20:00:00,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,concorde,2024-01-08 11:00:00 - 12:00:00,2024-02-08 11:00:00 - 12:00:00,2024-01-08 15:00:00 - 16:00:00,2024-07-30 20:00:00 - 21:00:00,2024-07-31 20:00:00 - 21:00:00,2024-01-08 20:00:00 - 21:00:00,2024-02-08 20:00:00 - 21:00:00,2024-03-08 20:00:00 - 21:00:00,2024-04-08 20:00:00 - 21:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,stade-roland-garros,2024-06-08 19:00:00 - 20:00:00,2024-07-08 19:00:00 - 20:00:00,2024-07-27 17:00:00 - 18:00:00,2024-07-28 17:00:00 - 18:00:00,2024-07-29 17:00:00 - 18:00:00,2024-07-30 17:00:00 - 18:00:00,2024-07-31 17:00:00 - 18:00:00,2024-01-08 17:00:00 - 18:00:00,2024-02-08 17:00:00 - 18:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,arena-paris-sud,2024-06-09 00:00:00 - 01:00:00,2024-07-09 00:00:00 - 01:00:00,2024-08-09 00:00:00 - 01:00:00,2024-09-09 00:00:00 - 01:00:00,2024-10-09 00:00:00 - 01:00:00,2024-07-27 12:00:00 - 13:00:00,2024-07-28 12:00:00 - 13:00:00,2024-07-29 12:00:00 - 13:00:00,2024-07-30 12:00:00 - 13:00:00,...,2024-10-08 14:00:00 - 15:00:00,2024-11-08 14:00:00 - 15:00:00,2024-07-08 18:00:00 - 19:00:00,2024-08-08 18:00:00 - 19:00:00,2024-09-08 18:00:00 - 19:00:00,2024-10-08 19:00:00 - 20:00:00,2024-07-08 22:00:00 - 23:00:00,2024-08-08 22:00:00 - 23:00:00,2024-09-08 22:00:00 - 23:00:00,2024-10-08 23:00:00 - 00:00:00
9,arena-bercy,2024-07-27 14:00:00 - 15:00:00,2024-07-28 13:00:00 - 14:00:00,2024-07-28 17:00:00 - 18:00:00,2024-05-08 15:00:00 - 16:00:00,2024-07-27 18:00:00 - 19:00:00,2024-07-28 20:00:00 - 21:00:00,2024-03-08 18:00:00 - 19:00:00,2024-04-08 17:00:00 - 18:00:00,2024-07-27 23:00:00 - 00:00:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


### TABLEAUX A EXPLOITER CI DESSUS ###